In [ ]:
import os

os.environ["RAY_ACCEL_ENV_VAR_OVERRIDE_ON_ZERO"] = "0"


import ray

ray.init("ray://localhost:10001")

In [ ]:
@ray.remote
def test2():
    import time

    time.sleep(1.0)
    print(456)
    print(789)


await test2.remote()

In [ ]:
import ray_log_notebook

ray_log_notebook.enable()

In [ ]:
import time


@ray.remote
def test():
    time.sleep(1.0)
    print(123)
    print(223)


await test.remote()

In [ ]:
@ray.remote
def test2():
    import time

    time.sleep(1.0)
    print(456)
    print(789)


await test2.remote()

In [ ]:
task = test.remote()

In [ ]:
task2 = test2.remote()